In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
#import missingno as msno 
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

In [3]:
system = 'linux'

# Prenatal Data
parent = os.path.dirname(os.getcwd())
if system == 'windows':
    dataPath = os.path.join(parent, r"Data\MOMI\Final_Prenatal_DeIdentified.xlsx")
    prenatal = pd.read_excel('file:\\' + dataPath)
else:
    dataPath = os.path.join(parent, r"Data/MOMI/Final_Prenatal_DeIdentified.xlsx")
    prenatal = pd.read_excel('file://' + dataPath)

# MOMI Data
parent = os.path.dirname(os.getcwd())
if system == 'windows':
    dataPath = os.path.join(parent, r"Data\MOMI\Final_MOMI_DeIdentified_Update_39Mar2021.xlsx")
    momi = pd.read_excel('file:\\' + dataPath)
else:
    dataPath = os.path.join(parent, r"Data/MOMI/Final_MOMI_DeIdentified_Update_39Mar2021.xlsx")
    momi = pd.read_excel('file://' + dataPath)

# Ultrasound Data
parent = os.path.dirname(os.getcwd())
if system == 'windows':
    dataPath = os.path.join(parent, r"Data\MOMI\Final_Ultrasound_DeIdentified.xlsx")
    ultrasound = pd.read_excel('file:\\' + dataPath)
else: 
    dataPath = os.path.join(parent, r"Data/MOMI/Final_Ultrasound_DeIdentified.xlsx")
    ultrasound = pd.read_excel('file://' + dataPath)

# Fix MOMI missing values to np.NaN
momi['MIDBV'] = np.where(momi['MIDBV'] == 99, np.NaN, momi['MIDBV'])
momi['MIDCHLAM'] = np.where(momi['MIDCHLAM'] == 99, np.NaN, momi['MIDCHLAM'])
momi['MIDCONDY'] = np.where(momi['MIDCONDY'] == 99, np.NaN, momi['MIDCONDY'])
momi['MIDGC'] = np.where(momi['MIDGC'] == 99, np.NaN, momi['MIDGC'])
momi['MIDHEPB'] = np.where(momi['MIDHEPB'] == 99, np.NaN, momi['MIDHEPB'])
momi['MIDTRICH'] = np.where(momi['MIDTRICH'] == 99, np.NaN, momi['MIDTRICH'])
momi['MIDGBS'] = np.where(momi['MIDGBS'] == 99, np.NaN, momi['MIDGBS'])
momi['MomEducation_State'] = np.where(momi['MomEducation_State'] == 'Unknown', np.NaN, momi['MomEducation_State'])
momi['DadEducation_State'] = np.where(momi['DadEducation_State'] == 'Unknown', np.NaN, momi['DadEducation_State'])
momi['Smoke_b'] = np.where(momi['Smoke_b'] == 'Unknown (unable to assess)', np.NaN, momi['Smoke_b'])
momi['Race'] = np.where(momi['Race'].isin(['9', 'A']), np.NaN, momi['Race'])
momi['Ethnicity'] = np.where(momi['Ethnicity'].isin(['UNSPECIFIED']), np.NaN, momi['Ethnicity'])
momi['InfSex'] = np.where(momi['InfSex'] == 'U', np.NaN, momi['InfSex'])
momi['InfSex'] = np.where(momi['InfSex'] == 'f', 'F', momi['InfSex'])


# Dropping erroneous prenatal data. This data does not actually exist, is thousands of missing values
prenatal.drop(prenatal[prenatal['DELWKSGT'].isnull()].index, inplace=True)
prenatal.drop(prenatal[prenatal['PNV_Total_Number'].isnull()].index, inplace=True)

"""
insuranceMap = {1: 'MedicalAssistance',
                2: 'PrivateInsurance',
                3: 'Self-pay'}

momi['DFC'] = momi['DFC'].map(insuranceMap)

outcomeMap = {1: 'SingleStillborn',
              2: 'TwinsLiveborn',
              3: 'TwinsOneLive',
              4: 'TwinsStillborn',
              5: 'MultsLiveborn',
              6: 'OtherMultSomeLive',
              7: 'OtherMultStillborn',
              9: np.NaN,
              10: 'SingleLiveborn'}

momi['MMULGSTD'] = momi['MMULGSTD'].map(outcomeMap)

neurMuscDiseaseMap = {0: 'None',
                      1: 'Multiple Sclerosis',
                      2: 'Cerebal Palsy',
                      3: 'Myotonic Dystrophy',
                      4: 'Myasthenia Gravis',
                      5: 'Other'}

momi['MCNSMUSC'] = momi['MCNSMUSC'].map(neurMuscDiseaseMap)

collagenVascMap = {0: 'None',
                   1: 'Rhematoid Arthritis',
                   2: 'Lupus',
                   8: 'Multiple Diagnostic Codes'}

momi['MCOLVASC'] = momi['MCOLVASC'].map(collagenVascMap)

struHeartMap = {0: 'None',
                1: 'Congenital Heart Disease',
                2: 'Rheumatic Heart Disease',
                3: 'Myocarditis/Cardiomyopathy',
                4: 'ValveDisorder',
                5: 'ArtificialValves',
                9: 'Other'}

momi['MCVDANAT'] = momi['MCVDANAT'].map(struHeartMap)

postpartMap = {0: 'None',
               1: 'Endometritis',
               2: 'UrinaryTractInfection',
               3: 'Hemmorrage',
               4: 'WoundInfection',
               5: 'Disseminated',
               6: 'Obstruction',
               9: 'Other'}

momi['MDELCOMP'] = momi['MDELCOMP'].map(postpartMap)

diabetesMap = {0: 'None',
               1: 'GestationalDiabetes',
               2: 'TypeI',
               3: 'TypeII',
               4: 'UnspecifiedPriorDiabetes'}

momi['MENDDIAB'] = momi['MENDDIAB'].map(diabetesMap)

thyroidMap = {0: 'None',
              1: 'Hyperthyroid',
              2: 'Hypothyroid',
              9: 'Other'}

momi['MENDTHY'] = momi['MENDTHY'].map(thyroidMap)

liverGallMap = {0: 'None',
                1: 'HepA',
                2: 'HepB',
                3: 'HepC',
                4: 'HepD',
                5: 'HepE',
                6: 'LiverTransplant',
                7: 'Cholelithiasis',
                8: 'Pancreatitis',
                9: 'Other'}

momi['MGILGBP'] = momi['MGILGBP'].map(liverGallMap)

kidneyMap = {0: 'None',
             1: 'Glomerulonephritis',
             2: 'Pyelonephritis;',
             3: 'LupusNephritis',
             4: 'NephroticSyndrome',
             5: 'Nephrolithiasis',
             6: 'Transplant;',
             7: 'RenalAbscess',
             8: 'MultipleDiagnosticCodes',
             9: 'Other'}

momi['MGURENAL'] = momi['MGURENAL'].map(kidneyMap)

anemiaMap = {0: 'None',
             1: 'IronDeficiencyAnemia',
             2: 'B12DeficiencyAnemia',
             3: 'FolateDeficiencyAnemia',
             9: 'UnspecifiedAnemia'}

momi['MHEMANEM'] = momi['MHEMANEM'].map(anemiaMap)

hemoGlob = {0: 'None',
            1: 'Hgb-SS',
            2: 'Hgb-SC',
            3: 'Hgb-Sthal',
            4: 'AlphaThalassemia',
            5: 'BetaThalassemia',
            6: 'SickleCellTrait',
            9: 'Other'}

momi['MHEMHGB'] = momi['MHEMHGB'].map(hemoGlob)

thromMap = {0: 'None',
            1: 'Gestational',
            2: 'DisseminatedIntravascularCoagulation',
            3: 'MultipleDiagnosticCodes',
            9: 'Other'}

momi['MHEMPLT'] = momi['MHEMPLT'].map(thromMap)

viralMap = {0: 'None',
            1: 'PrimaryCMV',
            2: 'ParovirusB19',
            3: 'Rubella',
            4: 'Toxoplasma',
            5: np.NaN,
            8: 'MultipleDiagnosticCodes',
            9: 'Other'}

momi['MIDVIRPR'] = momi['MIDVIRPR'].map(viralMap)

substanceMap = {0: 'None',
                1: 'Stimulants',
                2: 'Sedatives/Hypnotics/Anxiolytics',
                3: 'Anti-depressants/OtherPsychoactives',
                4: 'Hallucinogens',
                6: 'Alcohol',
                8: 'MultipleDiagnosticCodes',
                9: 'Other'}

momi['MTOXOTHR'] = momi['MTOXOTHR'].map(substanceMap)

anoAnoMap = {0: 'None',
             1: 'Anencephaly/Similar',
             2: 'Encephalocele',
             3: 'Microcephaly',
             4: 'CongenitalHydrocephalus',
             5: 'SpinaBifida',
             8: 'MultipleDiagnosticCodes',
             0: 'OtherCongenital'}

momi['ICNSANAT'] = momi['ICNSANAT'].map(anoAnoMap)

"""

# Ordinal Encoding Education
education_map = {'8th grade or less': 1,
                 '9th-12th grade, no diploma': 2,
                 'High school graduate or GED completed': 3,
                 'Some college credit, no degree': 4,
                 'Associate degree': 5,
                 "Bachelor's degree": 6,
                 "Master's degree": 7,
                 'Doctorate or professional degree': 8,
                 'Doctorate or Professional degree': 8}

momi['DadEducation_State'] = momi['DadEducation_State'].map(education_map)

momi['MomEducation_State'] = momi['MomEducation_State'].map(education_map)


# Renaming Race variables for easier comparison
raceMap = {'B': 'AfricanAmerican', 'C': "Chinese", 'D': "Declined",
           'E': "OtherAsian", 'F': "Filipino", 'G': "Guam/Chamorro",
           'I': "Indian(Asian)", 'J': "Japanese", 'K': "Korean",
           'L': "AlaskanNative", 'N': "NativeAmerican", 'P': "OtherPacificIslander",
           'Q': "Hawaiian", 'S': "Samoan", 'V': "Vietnamese", 'W': "White", 'D': "Declined", 9: np.NaN}

momi['Race'] = momi['Race'].map(raceMap)

# Collapsing Race categories
momi['RaceCollapsed'] = np.NaN

AsianGroups = ['OtherAsian', 'Indian(Asian)', 'Chinese', 'Korean', 'Filipino', 'Japanese', 'Vietnamese']
Polynesian = ['Hawaiian', 'Samoan', 'OtherPacificIslander', 'Guam/Chamorro']  # Unsure about Guam
NativeGroups = ['NativeAmerican', 'AlaskanNative']

# Asian
momi['RaceCollapsed'] = np.where((momi['Race'].isin(AsianGroups)), 'Asian', momi['RaceCollapsed'])
# Polynesian
momi['RaceCollapsed'] = np.where((momi['Race'].isin(Polynesian)), 'Polynesian', momi['RaceCollapsed'])
# Native
momi['RaceCollapsed'] = np.where((momi['Race'].isin(NativeGroups)), 'Native', momi['RaceCollapsed'])
# African
momi['RaceCollapsed'] = np.where((momi['Race'] == 'AfricanAmerican'), 'African', momi['RaceCollapsed'])
# White
momi['RaceCollapsed'] = np.where((momi['Race'] == 'White'), 'White', momi['RaceCollapsed'])

# Renaming Hypertensive variables for easier comparison
hypMap = {0: 'None', 1: 'TransientHypertension',
          2: 'Preeclampsia mild', 3: 'PreeclampsiaSevere',
          5: 'Eclampsia', 6: 'ChronicHypwPre',
          8: 'MultipleDiagnosticCodes', 9: 'UnspecifiedHyp'}

momi['MOBHTN'] = momi['MOBHTN'].map(hypMap)

# Set mildpe to 0 if marked severe
momi['Mild_PE'] = np.where(momi['MOBHTN'] == 'PreeclampsiaSevere', 0, momi['Mild_PE'])

# Looking at any occurance of Preeclampsia/Eclampsia
momi['Preeclampsia/Eclampsia'] = np.NaN
momi['Preeclampsia/Eclampsia'] = np.where(
    (momi['Mild_PE'] == 1) | (momi['Severe_PE'] == 1) | (momi['SIPE'] == 1) | (momi['MOBHTN'] == 'Eclampsia'), 1, 0)

# Renaming columns for easier analysis
momi.rename(columns={"DMOMAGE": "MotherAge", "FatherAge_State": "FatherAge", "DFC": "Insurance",
                     "DELWKSGT": "GestAgeDelivery", "MHXGRAV": "TotalNumPregnancies",
                     "MHXPARA": "DeliveriesPriorAdmission",
                     "MHXABORT": "TotalAbortions", "PRIMIP": "Primagrivada", "DMOMHGT": "MaternalHeightMeters",
                     "MOBRPWT": "PrePregWeight", "MOBADMWT": "WeightAtAdmission",
                     "FOBLABHR": "HoursLaborToDelivery",
                     "FOBROMHR": "HoursMembraneReptureDelivery", "CSREPEAT": "RepeatCesarean",
                     "FDELTYPE": "DeliveryMethod",
                     "MMULGSTD": "OutcomeOfDelivery", "FOBDEATH": "FetalDeath",
                     "MCNSMUSC": "MaternalNeuromuscularDisease",
                     "MCOLVASC": "MCollagenVascularDisease", "MCVDANAT": "MStructuralHeartDiseas",
                     "MCVDHTN": "ChronicHypertension",
                     "MOBHTN": "PregRelatedHypertension", "MDELCOMP": "MPostPartumComplications",
                     "MDEPRESS": "Depression",
                     "MENDDIAB": "DiabetesMellitus", "MENDTHY": "ThyroidDisease",
                     "MGIHYPER": "HyperemesisGravidarum",
                     "MGILGBP": "MLiverGallPanc", "MGUINFER": "HistoryInfertility", "MGURENAL": "KidneyDisease",
                     "MHEARTOPER": "OperationsOnHeartandPericardium", "MHEMANEM": "MAnemiaWOHemoglobinopathy",
                     "MHEMHGB": "MHemoglobinopathy", "MHEMPLT": "Thrombocytopenia", "MHEMTRAN": "TransfusionOfPRBC",
                     "MIDBV": "BacterialVaginosis", "MIDCHLAM": "Chlamydia", "MIDCONDY": "Condylomata",
                     "MIDGBS": "GroupBStrep", "MIDGC": "GonococcalInfection", "MIDHEPB": "HepBInfection",
                     "MIDHSV": "Herpes", "MIDTB": "Tuberculosis", "MIDTRICH": "Trichomonas",
                     "MIDVIRPR": "ViralOrProtoInf",
                     "MINTERINJ": "ThoraxAbPelvInjuries", "MMORTECLAMP": "Eclampsia",
                     "MMORTHEARTFAIL": "HeartFailure",
                     "MMORTRENAL": "AcuteRenalFailure", "MMORTSICKLECELL": "SickleCell",
                     "MOBPRECS": "PreviousCesarean",
                     "MPULASTH": "Asthma", "MTOXCOC": "Cocaine", "MTOXNARC": "Opioid",
                     "MTOXOTHR": "OtherSubstanceAbuse",
                     "MTOXTHC": "Marijuana", "IDEMBWT": "InfantWeightGrams", "IGROWTH": "GestWeightCompare",
                     "ICNSANAT": "CNSAbnormality", "IIDSYPH": "CongenitalSyphilis", "IIDUTI": "UTI",
                     "Alcohol_a": 'Drinks/Week'}, inplace=True)

# Dropping variables with more than 20% missing values
momi = momi.loc[:, momi.isnull().mean() < .20]

# Joining the momi data with the prenatal data - we want women who never had preeclampsia and first incidence of
# preeclampsia, nothing else
# Step 1, split systolic and diastolic
new = prenatal["PNV_BP"].str.split("/", n=1, expand=True)
prenatal["Systolic"] = new[0]
prenatal["Diastolic"] = new[1]
prenatal[["Systolic", "Diastolic"]] = prenatal[["Systolic", "Diastolic"]].apply(pd.to_numeric)




In [8]:
early

,MOMI_ID,Delivery_Number_Per_Mother,DeliveryYear,Has_Prenatal_Data,Has_Ultrasound_PlacLoc,MotherAge,FatherAge,MomEducation_State,DadEducation_State,NumCig3MoPriorToPreg_State,...,Infant_Number,InfSex,NICULOS,InfantWeightGrams,GestWeightCompare,CNSAbnormality,CongenitalSyphilis,UTI,RaceCollapsed,Preeclampsia/Eclampsia
0,11111167991974,1,1996,0,0,22.0,NaN,NaN,NaN,NaN,...,1,M,0.0,1200.0,1.0,OtherCongenital,0.0,0.0,White,1
6,11111891991995,1,2013,0,1,17.0,18.0,2.0,2.0,5.0,...,1,M,NaN,1902.0,0.0,OtherCongenital,0.0,0.0,White,0
31,11118321991979,1,2005,0,0,25.0,NaN,NaN,NaN,NaN,...,1,M,11.0,2355.0,0.0,OtherCongenital,0.0,0.0,White,0
34,11118747991982,1,2017,0,1,35.0,NaN,3.0,NaN,0.0,...,1,M,137.0,664.0,0.0,OtherCongenital,0.0,0.0,White,0
41,11119497991971,1,2005,0,0,33.0,NaN,NaN,NaN,NaN,...,1,M,1.0,728.0,0.0,OtherCongenital,0.0,0.0,White,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234886,99975983991987,2,2018,0,1,30.0,NaN,NaN,NaN,NaN,...,2,M,13.0,959.0,0.0,OtherCongenital,0.0,0.0,White,0
234900,99978782991985,1,2015,0,0,30.0,NaN,NaN,NaN,NaN,...,1,M,0.0,565.0,0.0,OtherCongenital,0.0,0.0,White,0
234953,99997347991972,1,2012,0,1,39.0,40.0,6.0,6.0,0.0,...,2,F,33.0,1548.0,0.0,OtherCongenital,0.0,0.0,White,1
234954,99997347991972,1,2012,0,1,39.0,40.0,6.0,6.0,0.0,...,1,M,22.0,1906.0,0.0,OtherCongenital,0.0,0.0,White,1


In [138]:
# MAP = (Sys + (2*Dias))/3
prenatal['MAP'] = np.NaN
prenatal['MAP'] = (prenatal['Systolic'] + (2* prenatal['Diastolic']))/3

In [139]:
prenatal[['Systolic','Diastolic','MAP']].head()

,Systolic,Diastolic,MAP
0,102.0,70.0,80.666667
1,120.0,60.0,80.000000
2,110.0,66.0,80.666667
3,110.0,70.0,83.333333
4,110.0,70.0,83.333333


In [9]:
# Step 2, make indicator variable
prenatal['High'] = np.where((prenatal['Systolic'] >= 130) | (prenatal['Diastolic'] >= 80), 1, 0)

# Step 3, make a cumulative sum to count how many times this person has had spikes
prenatal['Prev_highBP'] = prenatal.groupby(['MOMI_ID', 'Delivery_Number_Per_Mother'])['High'].cumsum().astype(int)

# Drop all women under 14 weeks from prenatal data
prenatal.drop(prenatal.loc[prenatal['PNV_GestAge'] > 14].index, inplace=True)
momi.sort_values('MOMI_ID', inplace=True)
uniquePregMomi = momi.drop_duplicates(subset=['MOMI_ID', 'Delivery_Number_Per_Mother'], keep='last')

prenatal.sort_values('PNV_GestAge', ascending=False, inplace=True)  # For preferenceing high bp
uniquePregPrenatal = prenatal.drop_duplicates(subset=['MOMI_ID', 'Delivery_Number_Per_Mother'], keep='first')
join = pd.merge(uniquePregMomi, uniquePregPrenatal, how='right')

# Removes duplicates, keeping only instances with Preeclampsia
join.sort_values('Preeclampsia/Eclampsia', ascending=False, inplace=True)
join = join.drop_duplicates(subset=['MOMI_ID'], keep='first')

In [10]:
early = join.loc[join['GestAgeDelivery']<34]

In [11]:
early

,MOMI_ID,Delivery_Number_Per_Mother,DeliveryYear,Has_Prenatal_Data,Has_Ultrasound_PlacLoc,MotherAge,FatherAge,MomEducation_State,DadEducation_State,NumCig3MoPriorToPreg_State,...,DELWKSGT,MMULGSTD,PNV_Total_Number,PNV_GestAge,PNV_BP,PNV_Weight_Oz,Systolic,Diastolic,High,Prev_highBP
7909,54821479991981,1,2017,1,0,35.0,34.0,2.0,NaN,0.0,...,33.714286,10,11.0,13.142857,121/57,3520.0,121.0,57.0,0,0
33240,22536693991986,1,2020,1,1,33.0,NaN,NaN,NaN,NaN,...,33.714286,10,9.0,9.142857,120/72,2280.0,120.0,72.0,0,0
12115,35316713991990,3,2018,1,0,28.0,NaN,NaN,NaN,NaN,...,33.428571,10,7.0,12.571429,106/72,3264.0,106.0,72.0,0,0
12116,69246932991985,2,2020,1,1,35.0,NaN,NaN,NaN,NaN,...,33.428571,10,5.0,12.571429,124/88,3104.0,124.0,88.0,1,1
3130,58838513991987,1,2015,1,1,28.0,36.0,6.0,6.0,0.0,...,31.285714,2,9.0,13.571429,118/80,2592.0,118.0,80.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12481,24665769991992,2,2016,1,0,23.0,24.0,2.0,NaN,0.0,...,32.000000,10,7.0,12.571429,95/54,1648.0,95.0,54.0,0,0
12569,57376427991998,1,2016,1,1,17.0,20.0,2.0,3.0,0.0,...,30.571429,10,7.0,12.571429,113/65,1708.0,113.0,65.0,0,0
12599,49519622991982,1,2016,1,1,33.0,33.0,7.0,5.0,0.0,...,27.571429,2,8.0,12.571429,106/78,2064.0,106.0,78.0,0,0
12511,45182171991994,1,2016,1,1,22.0,99.0,NaN,NaN,0.0,...,21.571429,10,5.0,12.571429,94/76,3168.0,94.0,76.0,0,1


# Ultrasound join

IMPORTANT: Don't drop the duplicate momi_id and delivery numbers. The placenta can move around over the course of a pregnancy, so it's important to get all the locations throughout the pregnancy first. This is done by a groupby and sum, plus a reset index. Once that's done then it's safe to drop the duplicates

In [4]:
# Sort by delivery year and gestational age
ultrasound.sort_values(by=['DeliveryYear', 'USGestAge'])
ultrasound = ultrasound.loc[ultrasound['USGestAge']<14]

In [5]:
locations = ['USPlacLoc_1', 'USPlacLoc_2','USPlacLoc_3', 'USPlacLoc_4', 'USPlacLoc_5', 'USPlacLoc_6']

# Step one: create indicator columns
ultrasound['Loc_Anterior'] = 0
ultrasound['Loc_Posterior'] = 0
ultrasound['Loc_Lateral'] = 0
ultrasound['Loc_Fundal'] = 0
ultrasound['Loc_Previa'] = 0
ultrasound['Loc_LowLying'] = 0
ultrasound['Loc_NonDiagnostic'] = 0

# For each location column: 
for col in locations:
    #Check what the position is and mark the appropriate binary column
    ultrasound['Loc_Anterior'] = np.where(((ultrasound[col] ==1)), 1,ultrasound['Loc_Anterior'])
    ultrasound['Loc_Posterior'] = np.where(((ultrasound[col] ==2)), 1,ultrasound['Loc_Posterior'])
    ultrasound['Loc_Lateral'] = np.where(((ultrasound[col] ==3)), 1,ultrasound['Loc_Lateral'])
    ultrasound['Loc_Lateral'] = np.where(((ultrasound[col] ==4)), 1,ultrasound['Loc_Lateral'])
    ultrasound['Loc_Fundal'] = np.where(((ultrasound[col] ==5)), 1,ultrasound['Loc_Fundal'])
    ultrasound['Loc_Previa'] = np.where(((ultrasound[col] ==6)), 1,ultrasound['Loc_Previa'])
    ultrasound['Loc_LowLying'] = np.where(((ultrasound[col] ==7)), 1,ultrasound['Loc_LowLying'])
    ultrasound['Loc_NonDiagnostic'] = np.where(((ultrasound[col] ==8)), 1,ultrasound['Loc_NonDiagnostic'])

ultrasound.drop(columns=locations, inplace=True)

In [6]:
ultragroup = ultrasound.groupby(['MOMI_ID','Delivery_Number_Per_Mother']).sum()
ultragroup.reset_index(level=['MOMI_ID', 'Delivery_Number_Per_Mother'], inplace=True)
ultragroup.drop_duplicates(subset=['MOMI_ID', 'Delivery_Number_Per_Mother'], inplace=True)

In [105]:
ultragroup.columns

Index(['MOMI_ID', 'Delivery_Number_Per_Mother', 'DeliveryYear', 'DELWKSGT',
       'MMULGSTD', 'SINGLETON', 'USGestAge', 'FetusNumber', 'Loc_Anterior',
       'Loc_Posterior', 'Loc_Lateral', 'Loc_Fundal', 'Loc_Previa',
       'Loc_LowLying', 'Loc_NonDiagnostic'],
      dtype='object')

## Testing merge

In [7]:
momi.drop_duplicates(subset=['MOMI_ID', 'Delivery_Number_Per_Mother'], inplace=True)

testMerge = pd.merge(momi,ultragroup[['MOMI_ID', 'Delivery_Number_Per_Mother', 'Loc_Anterior', 'Loc_Posterior',
       'Loc_Lateral', 'Loc_Fundal', 'Loc_Previa',
       'Loc_LowLying', 'Loc_NonDiagnostic']],on=['MOMI_ID', 'Delivery_Number_Per_Mother'], how='left')

In [8]:
testMerge.dropna(subset=['Loc_Anterior', 'Loc_Posterior',
       'Loc_Lateral', 'Loc_Fundal', 'Loc_Previa',
       'Loc_LowLying', 'Loc_NonDiagnostic'])

,MOMI_ID,Delivery_Number_Per_Mother,DeliveryYear,Has_Prenatal_Data,Has_Ultrasound_PlacLoc,MotherAge,Race,Insurance,GestAgeDelivery,TotalNumPregnancies,...,UTI,RaceCollapsed,Preeclampsia/Eclampsia,Loc_Anterior,Loc_Posterior,Loc_Lateral,Loc_Fundal,Loc_Previa,Loc_LowLying,Loc_NonDiagnostic
5,11111351991995,1,2016,1,1,21.0,AfricanAmerican,1.0,39.714286,1.0,...,0.0,African,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0
25,11116447991981,2,2013,0,1,32.0,White,2.0,39.000000,2.0,...,0.0,White,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
34,11118747991982,1,2017,0,1,35.0,White,1.0,24.000000,1.0,...,0.0,White,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
47,11122331991981,1,2014,0,1,33.0,White,2.0,38.000000,1.0,...,0.0,White,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0
49,11122648991978,1,2013,0,1,35.0,White,1.0,39.000000,1.0,...,0.0,White,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229466,99993799991985,1,2014,0,1,29.0,White,2.0,41.000000,1.0,...,0.0,White,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
229470,99995178991975,1,2016,1,1,41.0,White,2.0,39.000000,3.0,...,0.0,White,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
229474,99996362991982,1,2012,0,1,29.0,White,2.0,41.000000,1.0,...,0.0,White,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
229477,99996918991985,1,2014,0,1,28.0,White,2.0,39.000000,2.0,...,0.0,White,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [26]:
testMerge[['Loc_Anterior', 'Loc_Posterior',
       'Loc_Lateral', 'Loc_Fundal', 'Loc_Previa',
       'Loc_LowLying', 'Loc_NonDiagnostic']].sum()

Loc_Anterior         15311.0
Loc_Posterior        14275.0
Loc_Lateral           2092.0
Loc_Fundal            2422.0
Loc_Previa            4128.0
Loc_LowLying           943.0
Loc_NonDiagnostic    22247.0
dtype: float64

In [20]:
ultragroup[['Loc_Anterior', 'Loc_Posterior',
       'Loc_Lateral', 'Loc_Fundal', 'Loc_Previa',
       'Loc_LowLying', 'Loc_NonDiagnostic']].sum()

Loc_Anterior         15311
Loc_Posterior        14275
Loc_Lateral           2092
Loc_Fundal            2422
Loc_Previa            4128
Loc_LowLying           943
Loc_NonDiagnostic    22247
dtype: int64

In [9]:
# Dropping variables we won't be using
testMerge.drop(columns=['MOMI_ID', 'Delivery_Number_Per_Mother', 'InfantWeightGrams', 'Eclampsia',
                   'GestAgeDelivery', 'DeliveryMethod', 'FetalDeath', 'OutcomeOfDelivery', 'DeliveryMethod',
                   'PregRelatedHypertension', 'Mild_PE', 'SIPE', 'Has_Prenatal_Data',
                   'Has_Ultrasound_PlacLoc', 'NICULOS', 'InfantWeightGrams','GestWeightCompare',
                   'Race', 'DeliveryYear', 'MPostPartumComplications'], inplace=True)


# Renaming Hypertensive variables for easier comparison
hypMap = {'African': 0, 'Asian': 1,
          'Native': 2, 'Polynesian': 3,
          'White': 4}

testMerge['RaceCollapsed'] = testMerge['RaceCollapsed'].map(hypMap)

hypMap = {'F': 0, 'M': 1}
testMerge['InfSex'] = testMerge['InfSex'].map(hypMap)


testMerge.to_csv('momiUS_062521.csv', index=False)

In [10]:
for c in testMerge.columns:
    print(f'----{c}----')
    print(testMerge[c].value_counts().sort_index())
    print('')

----MotherAge----
11.0        3
12.0        5
13.0       51
14.0      227
15.0      716
16.0     1395
17.0     2460
18.0     3884
19.0     5550
20.0     6494
21.0     6941
22.0     7934
23.0     8001
24.0     8828
25.0     9841
26.0    10870
27.0    12517
28.0    13977
29.0    15321
30.0    15852
31.0    15807
32.0    14987
33.0    13513
34.0    12030
35.0    10455
36.0     8651
37.0     6763
38.0     5328
39.0     3918
40.0     2730
41.0     1840
42.0     1133
43.0      658
44.0      401
45.0      197
46.0      104
47.0       45
48.0       23
49.0       14
50.0       11
51.0        4
52.0        5
53.0        1
54.0        2
55.0        1
56.0        1
59.0        1
Name: MotherAge, dtype: int64

----Insurance----
1.0     53797
2.0    117375
3.0     58307
Name: Insurance, dtype: int64

----TotalNumPregnancies----
0.0        33
1.0     73812
2.0     64113
3.0     38753
4.0     20445
5.0     10528
6.0      5249
7.0      2649
8.0      1409
9.0       775
10.0      396
11.0      212
12.0  

0.0    24060
1.0    11615
2.0     1154
3.0       59
4.0       35
5.0        1
6.0        5
Name: Loc_Posterior, dtype: int64

----Loc_Lateral----
0.0    34933
1.0     1913
2.0       76
3.0        3
4.0        3
6.0        1
Name: Loc_Lateral, dtype: int64

----Loc_Fundal----
0.0    34618
1.0     2209
2.0       95
3.0        5
4.0        2
Name: Loc_Fundal, dtype: int64

----Loc_Previa----
0.0    32990
1.0     3760
2.0      173
3.0        3
4.0        2
5.0        1
Name: Loc_Previa, dtype: int64

----Loc_LowLying----
0.0    36004
1.0      907
2.0       18
Name: Loc_LowLying, dtype: int64

----Loc_NonDiagnostic----
0.0     18526
1.0     15379
2.0      2528
3.0       302
4.0       128
5.0        25
6.0        32
7.0         3
8.0         1
9.0         2
10.0        3
Name: Loc_NonDiagnostic, dtype: int64



## Old Code

Now I just need to deal with the multiples

In [40]:
# Drop all women under 14 weeks from prenatal data
USTest.drop(USTest.loc[USTest['USGestAge'] > 14].index, inplace=True)
USTest.sort_values(by='MOMI_ID')
#USTest.loc[USTest.duplicated(subset=['MOMI_ID','Delivery_Number_Per_Mother'])]
USTest

,MOMI_ID,DeliveryYear,Delivery_Number_Per_Mother,DELWKSGT,MMULGSTD,SINGLETON,USExamCategory,USExamType,USGestAge,USGestAgeType,FetusNumber,Loc_Anterior,Loc_Posterior,Loc_LeftLateral,Loc_RightLateral,Loc_Fundal,Loc_Previa,Loc_LowLying,Loc_NonDiagnostic
2,81155213991973,2011,1,37.000000,10,1,Obstetric,FirstTrimester,9.7,LMP,1,0,0,0,0,0,0,0,1
3,81155213991973,2011,1,37.000000,10,1,Obstetric,FirstTrimester,13.1,LMP,1,0,0,0,1,1,0,0,0
6,98174825991981,2014,2,38.000000,10,1,Obstetric,FirstTrimester,13.6,LMP,1,0,1,0,0,0,0,0,0
9,76494213991976,2012,1,30.000000,10,1,Obstetric,FirstTrimester,11.4,LMP,1,1,0,0,0,0,0,0,0
20,64227234991978,2012,1,39.000000,10,1,Obstetric,FirstTrimester,13.4,LMP,1,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221183,76739537991993,2020,1,21.285714,1,1,Obstetric,FirstTrimester,12.4,LMP,1,0,1,0,0,0,0,0,0
221248,58515446991985,2020,1,25.285714,10,1,Obstetric,FirstTrimester,12.3,Crown Rump,1,1,0,0,0,0,0,0,0
221249,58515446991985,2020,1,25.285714,10,1,Obstetric,FirstTrimester,13.1,PriorExam,1,1,0,0,0,0,0,0,0
221260,57247562991987,2016,1,39.714286,10,1,Obstetric,FirstTrimester,6.1,Crown Rump,1,0,0,0,0,0,0,0,1


In [41]:
USTest.loc[USTest['FetusNumber']!= 1]

,MOMI_ID,DeliveryYear,Delivery_Number_Per_Mother,DELWKSGT,MMULGSTD,SINGLETON,USExamCategory,USExamType,USGestAge,USGestAgeType,FetusNumber,Loc_Anterior,Loc_Posterior,Loc_LeftLateral,Loc_RightLateral,Loc_Fundal,Loc_Previa,Loc_LowLying,Loc_NonDiagnostic
23,91911455991972,2011,1,39.000000,2,0,Obstetric,FirstTrimester,11.9,Conception,2,1,0,0,0,0,0,0,0
158,43522444991979,2014,3,38.000000,2,0,Obstetric,FirstTrimester,13.4,LMP,2,0,1,0,0,0,0,0,0
189,14894948991980,2015,1,24.285714,2,0,Obstetric,FirstTrimester,7.1,LMP,2,0,0,0,0,0,0,0,1
191,14894948991980,2015,1,24.285714,2,0,Obstetric,FirstTrimester,11.6,LMP,2,0,1,0,0,0,0,0,0
392,68346721991982,2015,3,29.285714,2,0,Obstetric,FirstTrimester,7.7,LMP,2,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220222,42931443991996,2020,1,37.142857,2,0,Obstetric,FirstTrimester,9.0,LMP,2,0,1,0,0,0,0,0,0
220820,83844221991992,2020,1,24.714286,2,0,Obstetric,FirstTrimester,10.4,LMP,2,0,0,0,0,0,0,0,1
221007,74661382991987,2020,1,21.428571,10,1,Obstetric,FirstTrimester,5.9,Composite,2,0,0,0,0,0,0,0,1
221032,86937822991988,2020,1,29.857143,2,0,Obstetric,FirstTrimester,10.9,LMP,2,0,1,0,0,0,0,0,0


In [42]:
USTest.loc[USTest['MOMI_ID']== 43522444991979]

,MOMI_ID,DeliveryYear,Delivery_Number_Per_Mother,DELWKSGT,MMULGSTD,SINGLETON,USExamCategory,USExamType,USGestAge,USGestAgeType,FetusNumber,Loc_Anterior,Loc_Posterior,Loc_LeftLateral,Loc_RightLateral,Loc_Fundal,Loc_Previa,Loc_LowLying,Loc_NonDiagnostic
156,43522444991979,2011,2,38.0,10,1,Obstetric,FirstTrimester,13.6,Crown Rump,1,0,1,0,0,0,0,0,0
157,43522444991979,2014,3,38.0,2,0,Obstetric,FirstTrimester,13.4,LMP,1,0,1,0,0,0,0,0,0
158,43522444991979,2014,3,38.0,2,0,Obstetric,FirstTrimester,13.4,LMP,2,0,1,0,0,0,0,0,0


So we'll need to single out the non-singletons

In [43]:
USTest.loc[USTest['SINGLETON']==0]

,MOMI_ID,DeliveryYear,Delivery_Number_Per_Mother,DELWKSGT,MMULGSTD,SINGLETON,USExamCategory,USExamType,USGestAge,USGestAgeType,FetusNumber,Loc_Anterior,Loc_Posterior,Loc_LeftLateral,Loc_RightLateral,Loc_Fundal,Loc_Previa,Loc_LowLying,Loc_NonDiagnostic
22,91911455991972,2011,1,39.000000,2,0,Obstetric,FirstTrimester,11.9,Conception,1,1,0,0,0,0,0,0,0
23,91911455991972,2011,1,39.000000,2,0,Obstetric,FirstTrimester,11.9,Conception,2,1,0,0,0,0,0,0,0
157,43522444991979,2014,3,38.000000,2,0,Obstetric,FirstTrimester,13.4,LMP,1,0,1,0,0,0,0,0,0
158,43522444991979,2014,3,38.000000,2,0,Obstetric,FirstTrimester,13.4,LMP,2,0,1,0,0,0,0,0,0
188,14894948991980,2015,1,24.285714,2,0,Obstetric,FirstTrimester,7.1,LMP,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220820,83844221991992,2020,1,24.714286,2,0,Obstetric,FirstTrimester,10.4,LMP,2,0,0,0,0,0,0,0,1
221031,86937822991988,2020,1,29.857143,2,0,Obstetric,FirstTrimester,10.9,LMP,1,0,1,0,0,0,0,0,0
221032,86937822991988,2020,1,29.857143,2,0,Obstetric,FirstTrimester,10.9,LMP,2,0,1,0,0,0,0,0,0
221109,24154633991992,2020,1,34.714286,2,0,Obstetric,FirstTrimester,10.1,PatientEDC,1,1,0,0,0,0,0,0,0


They look like they might all be in the same locations. How do I check? 

In [53]:
USTest.sort_values(by=['MOMI_ID','Delivery_Number_Per_Mother'])
USTest.loc[USTest['SINGLETON']==0].groupby(['MOMI_ID','Delivery_Number_Per_Mother', 'USGestAge', 'FetusNumber'])['Loc_Anterior', 'Loc_Posterior',
       'Loc_LeftLateral', 'Loc_RightLateral', 'Loc_Fundal', 'Loc_Previa',
       'Loc_LowLying', 'Loc_NonDiagnostic'].sum()

<ipython-input-53-4ff6e56a97a5>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  USTest.loc[USTest['SINGLETON']==0].groupby(['MOMI_ID','Delivery_Number_Per_Mother', 'USGestAge', 'FetusNumber'])['Loc_Anterior', 'Loc_Posterior',


Loc_Anterior  \
MOMI_ID        Delivery_Number_Per_Mother USGestAge FetusNumber                 
11144674991986 1                          11.9      1                       0   
                                                    2                       0   
11154913991985 3                          11.9      1                       1   
                                                    2                       1   
11173443991989 1                          9.2       1                       0   
...                                                                       ...   
99553926991987 2                          11.7      2                       0   
99828932991981 1                          9.3       1                       1   
                                                    2                       0   
                                          12.3      1                       1   
                                                    2                       1   

                                                                 Loc_Posterior  \
MOMI_ID        Delivery_Number_Per_Mother USGestAge FetusNumber                  
11144674991986 1                          11.9      1                        1   
                                                    2                        1   
11154913991985 3                          11.9      1                        0   
                                                    2                        0   
11173443991989 1                          9.2       1                        1   
...                                                                        ...   
99553926991987 2                          11.7      2                        1   
99828932991981 1                          9.3       1                        0   
                                                    2                        1   
                                          12.3      1                        0   
                                                    2                        0   

                                                                 Loc_LeftLateral  \
MOMI_ID        Delivery_Number_Per_Mother USGestAge FetusNumber                    
11144674991986 1                          11.9      1                          0   
                                                    2                          0   
11154913991985 3                          11.9      1                          1   
                                                    2                          0   
11173443991989 1                          9.2       1                          0   
...                                                                          ...   
99553926991987 2                          11.7      2                          0   
99828932991981 1                          9.3       1                          0   
                                                    2                          0   
                                          12.3      1                          0   
                                                    2                          0   

                                                                 Loc_RightLateral  \
MOMI_ID        Delivery_Number_Per_Mother USGestAge FetusNumber                     
11144674991986 1                          11.9      1                           0   
                                                    2                           0   
11154913991985 3                          11.9      1                           0   
                                                    2                           0   
11173443991989 1                          9.2       1                           0   
...                                                                           ...   
99553926991987 2                          11.7      2                           0   
99828932991981 1                          9.3       1                           0   
 

In [54]:
multTest = ultrasound.copy()

In [65]:
multTest.sort_values(by=['USGestAge'])
multTest.drop(multTest.loc[multTest['USGestAge'] > 14].index, inplace=True)

In [57]:
multTest=multTest.loc[multTest['SINGLETON']==0]

In [66]:
multTest

,MOMI_ID,DeliveryYear,Delivery_Number_Per_Mother,DELWKSGT,MMULGSTD,SINGLETON,USExamCategory,USExamType,USGestAge,USGestAgeType,FetusNumber,USPlacLoc_1,USPlacLoc_2,USPlacLoc_3,USPlacLoc_4,USPlacLoc_5,USPlacLoc_6
22,91911455991972,2011,1,39.000000,2,0,Obstetric,FirstTrimester,11.9,Conception,1,1,NaN,NaN,NaN,NaN,NaN
23,91911455991972,2011,1,39.000000,2,0,Obstetric,FirstTrimester,11.9,Conception,2,1,NaN,NaN,NaN,NaN,NaN
157,43522444991979,2014,3,38.000000,2,0,Obstetric,FirstTrimester,13.4,LMP,1,2,NaN,NaN,NaN,NaN,NaN
158,43522444991979,2014,3,38.000000,2,0,Obstetric,FirstTrimester,13.4,LMP,2,2,NaN,NaN,NaN,NaN,NaN
188,14894948991980,2015,1,24.285714,2,0,Obstetric,FirstTrimester,7.1,LMP,1,8,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220820,83844221991992,2020,1,24.714286,2,0,Obstetric,FirstTrimester,10.4,LMP,2,8,NaN,NaN,NaN,NaN,NaN
221031,86937822991988,2020,1,29.857143,2,0,Obstetric,FirstTrimester,10.9,LMP,1,2,NaN,NaN,NaN,NaN,NaN
221032,86937822991988,2020,1,29.857143,2,0,Obstetric,FirstTrimester,10.9,LMP,2,2,NaN,NaN,NaN,NaN,NaN
221109,24154633991992,2020,1,34.714286,2,0,Obstetric,FirstTrimester,10.1,PatientEDC,1,1,NaN,NaN,NaN,NaN,NaN
